In [1]:
from copy import copy
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_iris, load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

Maybe I'll come back and fix the contour plot some day...

In [ ]:
# Set up: Add a bias term to X, initialize the parameters to random values, and set the learning rate & number of iterations
X_b = np.hstack([np.ones((100,1)), X])  # add a column of ones to X so we also calculate an intercept
theta = np.random.rand(2,1)
eta = 0.1
n_iter = 20

thetas = []
mses = []
y_preds = []

f, ((ax1, ax2), (ax3, _)) = plt.subplots(2, 2)
f.set_size_inches(12, 12)

# First plot: scatter plot of X & y, with fitted lines that turn more redd with each iteration
ax1.scatter(X, y, c='g')
ax1.set_title('Scatter of X and y')
ax1.set_ylabel('y')
ax2.set_xlabel('X')

# Set up the colour map for the fitted lines
cm = plt.get_cmap('bwr')
ax1.set_color_cycle([cm(i/n_iter) for i in range(n_iter)])

for i in range(n_iter):
    
    # Calculate the gradient and update the parameters
    gradient = (2/X.shape[0]) * X_b.T.dot(X_b.dot(theta) - y)
    theta -= eta * gradient
    
    # Calculate the predicted values for all observations, then calculate the MSE
    y_pred = X_b.dot(theta)
    mse = np.sum(((y_pred - y) ** 2)) / X.shape[0]  # ToDo: revisit this calculation
    
    # Plot the fitted line for this iteration in the first plot
    ax1.plot(X, y_pred)
    
    # Save the calculated values
    mses.append(copy(mse))
    thetas.append(copy(theta))
    y_preds.append(copy(y_pred))
    
# Second plot: The MSE with each iteration
ax2.plot(mses)
ax2.set_title('MSE with each iteration')
ax2.set_xlabel('Iteration')
ax2.set_ylabel('MSE')

# Third plot: contour plot of the loss function
thetas = np.squeeze(thetas)
x_lin = np.linspace(np.rint(np.min(thetas[:,0])), np.rint(np.max(thetas[:,0])))
y_lin = np.linspace(np.rint(np.min(thetas[:,1])), np.rint(np.max(thetas[:,1])))

x_lin, y_lin = np.meshgrid(x_lin, y_lin)
xy = np.array([x_lin.ravel(), y_lin.ravel()]).T
mse_lin = np.sum(X_b.dot(xy.T) - np.outer(y, np.ones(2500)), axis=0) / X.shape[0]
mse_lin = mse_lin.reshape(x_lin.shape)

ax3.contour(x_lin, y_lin, mse_lin, norm=LogNorm(vmin=1.0, vmax=1000.0), 
            levels=np.logspace(0, 3, 20))

f.tight_layout()